## Monte Carlo Option Pricing Example Using CLT Confidence Intervals

This script shows how to use approximate Central Limit Theorem (CLT) confidence intervals with Monte Carlo to price a financial derivative or option. An option gives the right, but not the obligation, to conduct some transaction at a future date.

## Late for a Date

This example demonstrates the confidence intervals for probabilities and quantiles assuming that you must take different modes of transportation to get to your date.

You leave your office and walk to the parking lot.  You then drive to Chinatown, park your car, and walk to MingHin restaurant to meet your friends.  

For illustration purposes, we assume that the times of each segment are distributed uniformly:

\begin{align*}
 T_1  &= \text{ time to walk to your car } \sim \mathcal{U}[4,7], \\
 T_2  &= \text{ time to drive to Chinatown } \sim \mathcal{U}[10,15], \\
 T_3  &= \text{ time to park your car } \sim \mathcal{U}[0,12], \\
 T_4  &= \text{ time to walk to Ming Hin } \sim \mathcal{U}[2,8], \\
 T_{\text{total}}  &= \text{ total travel time } = T_1 + T_2 + T_3 + T_4.
\end{align*}

All times are given in minutes.  We want to know 

 * what is the average time required to get to your date, 
 * what is the probability of needing more than \(35\) minutes to get to your date, and
 * what is the time you need to leave by to guarantee with \(98\%\) confidence that you will be on time.




In [38]:
# Import necessary packages
import numpy as np
from scipy.optimize import fsolve
from scipy import stats


### Simulating travel times
First we construct a function that generates IID travel times.  If $U_1,\ldots, U_4$ are IID $\mathcal{U}[0,1]$ random variables, then 

 \begin{align*}
 T_1  &= 4 + 3 U_1, \\
 T_2  &= 10 + 5 U_2, \\
 T_3  &= 12 U_3, \\
 T_4  &= 2 + 6 U_4, \\
 T_{\text{total}}  & = T_1 + T_2 + T_3 + T_4 = 16 + 3U_1 + 5U_2 + 12U_3 + 6 U_4.
 \end{align*}


In [41]:
Ttot = lambda n: 16+ np.sum(np.random.rand(n,4)* np.array([3,5,12,6]),axis = 1)

In [42]:
Ttot(2)

array([31.95914464, 23.13306622])

### The average travel time
 The average travel time can be easily calculated analytically: 

$$
 \mu  = \mathbb{E}(T_{\text{total}}) = 16 + (3 + 5 + 12 + 6) \mathbb{E}(U)
 = 16 + 26 \times 1/2 = 29.
$$

Monte Carlo methods can be used to approximate this as well:


In [31]:
absTol = 0.01
relTol = 0
alpha = 0.01
muhat,nSamples = meanMC_CLT(Ttot,absTol,relTol,alpha) #IID Monte Carlo with absolute error of 0.01


In [34]:
print("The average travel time is about ",f"{muhat:6.2f}", "minutes.")

The average travel time is about   29.00 minutes.


### The probability of being late
If we only leave our office $29$ minutes before our date, we will sometimes be early and sometimes be late. 

In [112]:
n = 10000 # number of samples
lateTime = 29 # what time is considered late
Ttotval = Ttot(n)  # sample of n total times
exactCI,CLTCI = binomialCI(n,sum(Ttotval > lateTime),alpha) # confidence interval for being too late

In [113]:
print("If we leave our office", lateTime, "minutes before our date, the probability of being late is between", exactCI)

If we leave our office 29 minutes before our date, the probability of being late is between [0.49337135 0.51922263]


Since this is an important date we plan to leave $35$ minutes beforehand and hope to be early or on time. What is the chance that we will be late?
 
We now perform $n$ trials, and count the number of late dates.  Then we use *binomialCI* to compute a confidence interval on the probability of
being late for the date given that you leave $35$ minutes beforehand.

In [126]:
lateTime = 35 # what time is considered late
Ttotval = Ttot(n)  # sample of n total times
exactCI,CLTCI = binomialCI(n,sum(Ttotval > lateTime),alpha) # confidence interval for being too late

In [127]:
print("If we leave our office", lateTime, "minutes before our date, the probability of being late is between", exactCI)

If we leave our office 38 minutes before our date, the probability of being late is between [0.00718527 0.01229382]


The chance is of being late is around $8\%$.

### The cut-off time for being on time with a high probability
A related problem is determining how late we can leave our office and still get to the restaurant on time with a high probability, say, $98\%$.  Now we use *quantileCI* as follows:

In [133]:
p = 0.02 # the probability of lateness that can be tolerated
extremeVal = np.array([16,42])  #the extreme values of the distribution
lateDateQuantCI = quantileCI(1-p,Ttotval,extremeVal,alpha) # 1-p quantile for total travel times

In [134]:
print("If we leave our office around ", f"{lateDateQuantCI[0]:5.2f}", " to ", f"{lateDateQuantCI[1]:5.2f}", "minutes before our date, the probability of being late is about", p*100, "%")

If we leave our office around  36.95  to  37.39 minutes before our date, the probability of being late is about 2.0 %


You need to allow $38$ minutes to make sure that you will be on time with at least a $98\%$ probability.

#### meanMC_CLT function

In [9]:
def meanMC_CLT(inputRandomFuc,absTol,relTol,alpha): #Simple version of meanMC_CLT
    nsig = 1000 # initial size of the sample
    inflate = 1.2 # inflation rate
    YY = inputRandomFuc(nsig)
    sigma  = np.std(YY,ddof = 1)
    hum = np.mean(YY)
    sigmaUpBound = sigma * inflate #upper ound on the standard deviation
    nmu = max(1, np.power(np.ceil(stats.norm.ppf(1-alpha/2)*sigmaUpBound/max(absTol,relTol)),2).astype(int) )  # number of samples needed for the error tolerance
    mu = np.mean(inputRandomFuc(nmu))
    nSample = nsig + nmu
    return mu, nSample

#### binomialCI function

In [73]:
def binomialCI(ntot,nsuc,alpha):
   obsprob=nsuc/ntot # observed probability of success

   al2=alpha/2 # half of uncertainty
   if nsuc==0: # no successes observed
      plo=0  # the lower bound must be zero
   else: # use a nonlinear equation solver
      plo=fsolve(lambda x: stats.binom.cdf(nsuc-1,ntot,x)-1+al2, obsprob)[0]

   if nsuc==ntot: #no failures observed 
      pup=1 # the upper bound must be one
   else: # use a nonlinear equation solver
      pup=fsolve(lambda x: stats.binom.cdf(nsuc,ntot,x)-al2,obsprob)[0] # nonlinear equation
   
   exactci=np.array([plo,pup]) #confidence interval based on exact probabilities
   cltci= obsprob+ np.array([-1,1])*(-stats.norm.ppf(al2)*np.sqrt(obsprob*(1-obsprob))/np.sqrt(ntot))
   return exactci,cltci

#### quantileCI function

In [111]:
def quantileCI(quant,Xsample,extremes,alpha):
   n = len(Xsample) # number of samples
   Xorder = np.zeros(n+2)
   Xorder[0] = extremes[0]
   Xorder[1:n+1] = sorted(Xsample)
   Xorder[n+1] = extremes[1]
   al2 = alpha/2 # half significance level
   lo = 1+stats.binom.ppf(al2,n,quant); # position of the lower bound
   up = 2+stats.binom.ppf(1-al2,n,quant); # position of the upper bound
   quantci=np.array([Xorder[lo.astype(int)],Xorder[up.astype(int)]]) #confidence interval for quantile
   return quantci